# Основные библиотеки
!pip install pybind11
!pip install pandas openpyxl numpy
# Для возможности использовать transformers (опционально)
!pip install transformers torch

# Для дополнительных возможностей
!pip install scikit-learn 
!pip install gensim 

# Библиотеки для русского языка
!pip install razdel
!pip install pymorphy3

# Для Dostoevsky нужна модель
!pip install dostoevsky --no-deps
!pip install fasttext-wheel==0.9.2
DOSTOEVSKY_AVAILABLE = False



In [11]:
"""
Анализатор упоминаний для мониторинга репутации брендов
Использует специализированные библиотеки для русского языка
"""

import re
import pandas as pd
import numpy as np
import os
import sys
import warnings
from typing import List, Dict, Optional, Tuple
import chardet
warnings.filterwarnings('ignore')

# Проверка и установка недостающих библиотек
def install_missing_packages():
    """Установка недостающих пакетов"""
    required_packages = {
        'pandas': 'pandas',
        'numpy': 'numpy',
        'openpyxl': 'openpyxl',
        'xlrd': 'xlrd>=1.2.0',
        'chardet': 'chardet',
        'tqdm': 'tqdm'
    }
    
    missing = []
    for package, install_name in required_packages.items():
        try:
            __import__(package)
        except ImportError:
            missing.append(install_name)
    
    if missing:
        print(f"Обнаружены отсутствующие пакеты: {', '.join(missing)}")
        response = input("Установить их автоматически? (y/n): ")
        if response.lower() == 'y':
            import subprocess
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install"] + missing)
                print("Пакеты успешно установлены!")
            except Exception as e:
                print(f"Ошибка при установке пакетов: {e}")
                print("Пожалуйста, установите их вручную:")
                print(f"pip install {' '.join(missing)}")
        else:
            print("Продолжаем без установки...")

# Вызываем проверку пакетов
install_missing_packages()


class ModelDownloader:
    """Класс для скачивания и управления моделями"""
    
    @staticmethod
    def download_dostoevsky_model():
        """Скачивание модели Dostoevsky"""
        print("Попытка скачать модель Dostoevsky...")
        try:
            import requests
            import zipfile
            import tempfile
            import shutil
            
            # URL модели Dostoevsky
            model_url = "https://github.com/bureaucratic-labs/dostoevsky/raw/master/dostoevsky/data/models/fasttext-social-network-model.bin"
            
            # Создаем временную директорию
            temp_dir = tempfile.mkdtemp()
            model_path = os.path.join(temp_dir, "fasttext-social-network-model.bin")
            
            # Скачиваем модель
            print(f"Скачивание модели Dostoevsky...")
            response = requests.get(model_url, stream=True)
            total_size = int(response.headers.get('content-length', 0))
            
            if response.status_code == 200:
                with open(model_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                # Копируем в нужную директорию
                import dostoevsky
                dostoevsky_dir = os.path.dirname(dostoevsky.__file__)
                target_dir = os.path.join(dostoevsky_dir, 'data', 'models')
                os.makedirs(target_dir, exist_ok=True)
                target_path = os.path.join(target_dir, 'fasttext-social-network-model.bin')
                
                shutil.copy(model_path, target_path)
                print(f"Модель Dostoevsky успешно скачана в: {target_path}")
                return True
            else:
                print(f"Ошибка скачивания модели: {response.status_code}")
                return False
                
        except Exception as e:
            print(f"Ошибка при скачивании модели Dostoevsky: {e}")
            print("Используем альтернативную модель...")
            return False
    
    @staticmethod
    def check_and_fix_dostoevsky():
        """Проверка и исправление Dostoevsky"""
        try:
            from dostoevsky.tokenization import RegexTokenizer
            from dostoevsky.models import FastTextSocialNetworkModel
            
            # Пробуем загрузить модель
            tokenizer = RegexTokenizer()
            model = FastTextSocialNetworkModel(tokenizer=tokenizer)
            
            # Проверяем работу на тестовом тексте
            test_result = model.predict(["тестовый текст"], k=2)
            
            print("✓ Dostoevsky проверен и работает корректно")
            return True
            
        except Exception as e:
            print(f"Dostoevsky требует настройки: {str(e)[:100]}")
            
            # Пробуем скачать модель
            return ModelDownloader.download_dostoevsky_model()

In [12]:
class FileReader:
    """Класс для чтения файлов разных форматов"""
    
    @staticmethod
    def detect_file_encoding(file_path: str) -> Tuple[str, float]:
        """Определение кодировки файла"""
        try:
            with open(file_path, 'rb') as f:
                raw_data = f.read(100000)  # Читаем первые 100КБ для определения кодировки
                result = chardet.detect(raw_data)
                return result['encoding'], result['confidence']
        except Exception as e:
            print(f"Ошибка определения кодировки: {e}")
            return 'utf-8', 0.0
    
    @staticmethod
    def detect_file_format(file_path: str) -> str:
        """Определение формата файла"""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Файл не найден: {file_path}")
        
        # По расширению
        ext = os.path.splitext(file_path)[1].lower()
        
        if ext in ['.xlsx', '.xls']:
            return 'excel'
        elif ext in ['.csv', '.txt']:
            # Читаем первые несколько строк для определения
            try:
                encoding, confidence = FileReader.detect_file_encoding(file_path)
                with open(file_path, 'r', encoding=encoding, errors='ignore') as f:
                    first_lines = [f.readline() for _ in range(5)]
                
                # Проверяем наличие разделителей
                for line in first_lines:
                    if '|' in line and line.count('|') > 2:
                        return 'csv_pipe'
                    elif ';' in line and line.count(';') > 2:
                        return 'csv_semicolon'
                    elif '\t' in line and line.count('\t') > 2:
                        return 'csv_tab'
                    elif ',' in line and line.count(',') > 2:
                        return 'csv_comma'
                
                return 'csv_auto'
            except:
                return 'csv_auto'
        else:
            return 'unknown'
    
    @staticmethod
    def read_file(file_path: str) -> pd.DataFrame:
        """Чтение файла любого формата"""
        print(f"Чтение файла: {file_path}")
        
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Файл не найден: {file_path}")
        
        file_format = FileReader.detect_file_format(file_path)
        print(f"Определен формат: {file_format}")
        
        try:
            if file_format == 'excel':
                # Пробуем разные движки для Excel
                try:
                    df = pd.read_excel(file_path, engine='openpyxl')
                    print("✓ Прочитано с помощью openpyxl")
                except Exception as e1:
                    print(f"openpyxl не сработал: {e1}")
                    try:
                        df = pd.read_excel(file_path, engine='xlrd')
                        print("✓ Прочитано с помощью xlrd")
                    except Exception as e2:
                        print(f"xlrd не сработал: {e2}")
                        # Пробуем как CSV с автодетектом
                        encoding, _ = FileReader.detect_file_encoding(file_path)
                        df = pd.read_csv(file_path, encoding=encoding, engine='python')
                        print("✓ Прочитано как CSV (резервный метод)")
            
            elif file_format == 'csv_pipe':
                encoding, _ = FileReader.detect_file_encoding(file_path)
                df = pd.read_csv(file_path, sep='|', encoding=encoding, engine='python', on_bad_lines='skip')
            
            elif file_format == 'csv_semicolon':
                encoding, _ = FileReader.detect_file_encoding(file_path)
                df = pd.read_csv(file_path, sep=';', encoding=encoding, engine='python', on_bad_lines='skip')
            
            elif file_format == 'csv_tab':
                encoding, _ = FileReader.detect_file_encoding(file_path)
                df = pd.read_csv(file_path, sep='\t', encoding=encoding, engine='python', on_bad_lines='skip')
            
            elif file_format == 'csv_comma':
                encoding, _ = FileReader.detect_file_encoding(file_path)
                df = pd.read_csv(file_path, sep=',', encoding=encoding, engine='python', on_bad_lines='skip')
            
            else:  # csv_auto или unknown
                encoding, _ = FileReader.detect_file_encoding(file_path)
                # Пробуем разные разделители
                for sep in [',', ';', '\t', '|']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding=encoding, 
                                        engine='python', on_bad_lines='skip')
                        print(f"✓ Прочитано с разделителем '{sep}'")
                        break
                    except:
                        continue
                else:
                    # Последняя попытка: без указания разделителя
                    df = pd.read_csv(file_path, encoding=encoding, engine='python', 
                                   on_bad_lines='skip')
            
            # Очистка данных
            df = FileReader.clean_dataframe(df)
            
            print(f"✓ Успешно прочитано {len(df)} строк")
            return df
            
        except Exception as e:
            print(f"Ошибка при чтении файла: {e}")
            
            # Последняя попытка: прочитать как бинарный и преобразовать
            try:
                with open(file_path, 'rb') as f:
                    content = f.read()
                
                # Пробуем разные кодировки
                for encoding in ['utf-8', 'cp1251', 'cp1252', 'latin1', 'koi8-r']:
                    try:
                        text = content.decode(encoding)
                        lines = text.split('\n')
                        
                        # Создаем DataFrame из строк
                        data = []
                        for line in lines[:1000]:  # Ограничиваем первые 1000 строк
                            if line.strip():
                                data.append([line.strip()])
                        
                        df = pd.DataFrame(data, columns=['Текст'])
                        print(f"✓ Прочитано вручную с кодировкой {encoding}")
                        return df
                    except:
                        continue
            except Exception as e2:
                print(f"Все попытки чтения не удались: {e2}")
            
            # Создаем пустой DataFrame если ничего не получилось
            print("⚠ Создаем пустой DataFrame")
            return pd.DataFrame(columns=['Текст'])
    
    @staticmethod
    def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
        """Очистка DataFrame"""
        # Удаляем полностью пустые столбцы
        df = df.dropna(axis=1, how='all')
        
        # Удаляем полностью пустые строки
        df = df.dropna(how='all')
        
        # Преобразуем все столбцы в строки
        for col in df.columns:
            df[col] = df[col].astype(str).str.strip()
        
        # Ищем колонку с текстом
        text_columns = []
        for col in df.columns:
            col_lower = col.lower()
            if any(keyword in col_lower for keyword in ['текст', 'text', 'сообщение', 'message', 'комментарий']):
                text_columns.append(col)
            elif df[col].str.len().mean() > 20:  # Средняя длина > 20 символов
                text_columns.append(col)
        
        if text_columns:
            # Используем первую найденную колонку с текстом
            df = df.rename(columns={text_columns[0]: 'Текст'})
            print(f"✓ Найдена колонка с текстом: '{text_columns[0]}'")
        else:
            # Создаем колонку Текст из всех столбцов
            if len(df.columns) > 1:
                df['Текст'] = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)
                print("✓ Создана колонка 'Текст' из всех данных")
            elif len(df.columns) == 1:
                df = df.rename(columns={df.columns[0]: 'Текст'})
                print(f"✓ Переименована колонка '{df.columns[0]}' в 'Текст'")
        
        return df

In [13]:
class SentimentAnalyzer:
    """Улучшенный анализатор тональности с автовыбором моделей"""
    
    def __init__(self, use_dostoevsky: bool = True):
        self.models = {}
        self.model_priority = []
        
        # Инициализируем доступные модели
        self._init_models(use_dostoevsky)
    
    def _init_models(self, use_dostoevsky: bool):
        """Инициализация всех доступных моделей"""
        print("\nИнициализация моделей анализа тональности...")
        
        # 1. Dostoevsky (если требуется и доступен)
        if use_dostoevsky:
            dostoevsky_loaded = self._init_dostoevsky()
            if dostoevsky_loaded:
                self.models['dostoevsky'] = self.dostoevsky_analyze
                self.model_priority.append('dostoevsky')
        
        # 2. Transformers (ruBERT) - основной резерв
        transformers_loaded = self._init_transformers()
        if transformers_loaded:
            self.models['transformers'] = self.transformers_analyze
            self.model_priority.append('transformers')
        
        # 3. Простая модель на основе ключевых слов (всегда доступна)
        self.models['basic'] = self.basic_analyze
        self.model_priority.append('basic')
        
        print(f"✓ Доступно моделей: {len(self.models)} ({', '.join(self.models.keys())})")
        print(f"  Порядок приоритета: {', '.join(self.model_priority)}")
    
    def _init_dostoevsky(self) -> bool:
        """Инициализация Dostoevsky"""
        try:
            # Пробуем исправить Dostoevsky перед загрузкой
            if not ModelDownloader.check_and_fix_dostoevsky():
                print("  ✗ Dostoevsky: не удалось исправить/загрузить модель")
                return False
            
            from dostoevsky.tokenization import RegexTokenizer
            from dostoevsky.models import FastTextSocialNetworkModel
            
            self.dostoevsky_tokenizer = RegexTokenizer()
            self.dostoevsky_model = FastTextSocialNetworkModel(tokenizer=self.dostoevsky_tokenizer)
            
            # Тестируем модель
            test_text = "Это тестовое сообщение для проверки модели."
            test_result = self.dostoevsky_model.predict([test_text], k=2)
            
            print("  ✓ Dostoevsky: модель загружена и работает")
            return True
            
        except Exception as e:
            print(f"  ✗ Dostoevsky: ошибка загрузки - {str(e)[:100]}")
            return False
    
    def _init_transformers(self) -> bool:
        """Инициализация Transformers"""
        try:
            from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
            
            print("  Загрузка Transformers (ruBERT)...")
            
            # Используем более легкую модель если основная не работает
            model_name = "blanchefort/rubert-base-cased-sentiment"
            
            try:
                self.transformers_pipeline = pipeline(
                    "sentiment-analysis",
                    model=model_name,
                    tokenizer=model_name,
                    framework="pt"
                )
                print("  ✓ Transformers: основная модель загружена")
            except:
                # Резервная модель
                print("  Основная модель не загрузилась, пробуем резервную...")
                model_name = "seara/rubert-base-cased-ru-go-emotions"
                self.transformers_pipeline = pipeline(
                    "sentiment-analysis",
                    model=model_name,
                    tokenizer=model_name,
                    framework="pt"
                )
                print("  ✓ Transformers: резервная модель загружена")
            
            return True
            
        except Exception as e:
            print(f"  ✗ Transformers: ошибка загрузки - {str(e)[:100]}")
            return False
    
    def dostoevsky_analyze(self, text: str) -> Dict:
        """Анализ тональности с помощью Dostoevsky"""
        if not hasattr(self, 'dostoevsky_model'):
            raise ValueError("Dostoevsky модель не инициализирована")
        
        try:
            results = self.dostoevsky_model.predict([text], k=5)[0]
            
            sentiment_map = {
                'positive': 'Позитивная',
                'negative': 'Негативная', 
                'neutral': 'Нейтральная',
                'skip': 'Нейтральная',
                'speech': 'Нейтральная'
            }
            
            if results:
                main_sentiment = max(results.items(), key=lambda x: x[1])
                sentiment_label = sentiment_map.get(main_sentiment[0], 'Нейтральная')
                
                return {
                    'sentiment': sentiment_label,
                    'confidence': float(main_sentiment[1]),
                    'source': 'dostoevsky',
                    'details': results
                }
        except Exception as e:
            print(f"  Ошибка Dostoevsky при анализе: {e}")
        
        return None
    
    def transformers_analyze(self, text: str) -> Dict:
        """Анализ тональности с помощью Transformers"""
        if not hasattr(self, 'transformers_pipeline'):
            raise ValueError("Transformers модель не инициализирована")
        
        try:
            result = self.transformers_pipeline(text[:512])[0]  # Ограничиваем длину
            
            sentiment_map = {
                'POSITIVE': 'Позитивная',
                'NEGATIVE': 'Негативная',
                'NEUTRAL': 'Нейтральная',
                'LABEL_0': 'Негативная',  # Для некоторых моделей
                'LABEL_1': 'Позитивная',
                'LABEL_2': 'Нейтральная'
            }
            
            sentiment = sentiment_map.get(result['label'], 'Нейтральная')
            
            return {
                'sentiment': sentiment,
                'confidence': float(result['score']),
                'source': 'transformers',
                'details': result
            }
        except Exception as e:
            print(f"  Ошибка Transformers при анализе: {e}")
        
        return None
    
    def basic_analyze(self, text: str) -> Dict:
        """Базовый анализ по ключевым словам"""
        if not text:
            return {
                'sentiment': 'Нейтральная',
                'confidence': 0.5,
                'source': 'basic'
            }
        
        text_lower = text.lower()
        
        # Словари для русского языка
        positive_patterns = [
            r'\b(отличн|прекрасн|замечательн|супер|класс|лучш|хорош|рекомендую|советую|спасибо|благодарю|доволен|рад)\b',
            r'[😀😃😄😁😆😍🤩👍❤️😊🙂💪]',
            r'\b(люблю|обожаю|нравится|восхищаюсь|одобряю|хвалю)\b'
        ]
        
        negative_patterns = [
            r'\b(плох|ужасн|кошмар|отвратительн|некачествен|бадяж|брак|обман|развод|жульнич|воруют|жалоб|недоволен|возмущен|разочарован)\b',
            r'[😠😡🤬😢😭😤👎💔😞😣😫😩😒]',
            r'\b(протестую|требую|верните|накажу|проблем|сломал|испортил|не\s+работает)\b'
        ]
        
        # Подсчет паттернов
        pos_count = sum(1 for pattern in positive_patterns 
                       if re.search(pattern, text_lower, re.IGNORECASE))
        neg_count = sum(1 for pattern in negative_patterns 
                       if re.search(pattern, text_lower, re.IGNORECASE))
        
        # Эмоциональные маркеры
        if '😔' in text or '😟' in text or '😕' in text or '☹️' in text:
            neg_count += 1
        if '😂' in text or '🤣' in text or '😎' in text or '🤗' in text:
            pos_count += 1
        
        # Определение тональности
        if pos_count > neg_count and pos_count > 0:
            confidence = min(0.5 + (pos_count - neg_count) * 0.1, 0.9)
            return {'sentiment': 'Позитивная', 'confidence': confidence, 'source': 'basic'}
        elif neg_count > pos_count and neg_count > 0:
            confidence = min(0.5 + (neg_count - pos_count) * 0.1, 0.9)
            return {'sentiment': 'Негативная', 'confidence': confidence, 'source': 'basic'}
        elif pos_count == neg_count and pos_count > 0:
            return {'sentiment': 'Смешанная', 'confidence': 0.5, 'source': 'basic'}
        else:
            return {'sentiment': 'Нейтральная', 'confidence': 0.5, 'source': 'basic'}
    
    def analyze(self, text: str) -> Dict:
        """Анализ тональности с использованием лучшей доступной модели"""
        if not text or len(text.strip()) < 3:
            return {
                'sentiment': 'Нейтральная',
                'confidence': 0.5,
                'source': 'none'
            }
        
        # Пробуем модели в порядке приоритета
        for model_name in self.model_priority:
            if model_name in self.models:
                try:
                    result = self.models[model_name](text)
                    if result:
                        return result
                except Exception as e:
                    print(f"  Модель {model_name} не сработала: {e}")
                    continue
        
        # Если все модели не сработали, используем базовую
        return self.basic_analyze(text)

In [14]:
class MentionAnalyzer:
    """Основной анализатор упоминаний"""
    
    # Предопределенные теги
    UNIVERSAL_TAGS = {
        'quality': 'Качество продукта/услуги',
        'comparison': 'Сравнение с конкурентами',
        'complaint': 'Жалоба/проблема',
        'positive': 'Положительный отзыв',
        'question': 'Вопрос/справка',
        'recommendation': 'Рекомендация/совет',
        'service': 'Сервис/обслуживание',
        'price': 'Вопросы цены',
        'technical': 'Технические вопросы',
        'safety': 'Вопросы безопасности',
        'infrastructure': 'Инфраструктура',
        'corporate': 'Корпоративная информация'
    }
    
    def __init__(self, config: Dict):
        """
        Args:
            config: Конфигурация анализатора
        """
        self.object_name = config.get('object_name', 'Объект')
        self.keywords = [kw.lower().strip() for kw in config.get('keywords', [])]
        self.risk_words = [rw.lower().strip() for rw in config.get('risk_words', [])]
        self.positive_words = [pw.lower().strip() for pw in config.get('positive_words', [])]
        self.competitors = [c.lower().strip() for c in config.get('competitors', [])]
        
        # Инициализация анализатора тональности
        print(f"\nИнициализация анализатора для: {self.object_name}")
        self.sentiment_analyzer = SentimentAnalyzer(
            use_dostoevsky=config.get('use_dostoevsky', True)
        )
        
        # Статистика
        self.stats = {
            'total_mentions': 0,
            'relevant_mentions': 0,
            'negative_mentions': 0,
            'dangerous_mentions': 0,
            'sentiment_sources': {}
        }
    
    def is_relevant(self, text: str) -> bool:
        """Определение релевантности"""
        if not text or len(text.strip()) < 5:
            return False
        
        text_lower = text.lower()
        
        # Проверка ключевых слов
        for keyword in self.keywords:
            if keyword in text_lower:
                return True
        
        # Проверка частичных совпадений
        words = re.findall(r'\b\w+\b', text_lower)
        for word in words:
            for keyword in self.keywords:
                if keyword in word or word in keyword:
                    return True
        
        return False
    
    def extract_tags(self, text: str) -> List[str]:
        """Извлечение тегов"""
        tags = []
        text_lower = text.lower()
        
        # 1. Жалобы (высший приоритет)
        if re.search(r'жалоб|недовол|возмущ|протест|претенз|требую|верните', text_lower):
            tags.append(self.UNIVERSAL_TAGS['complaint'])
        
        # 2. Качество
        if re.search(r'качеств|некачествен|плох|ужас|кошмар|бадяж', text_lower):
            tags.append(self.UNIVERSAL_TAGS['quality'])
        
        # 3. Позитивные отзывы
        if re.search(r'отличн|прекрасн|замечательн|хорош|рекомендую|спасибо', text_lower):
            tags.append(self.UNIVERSAL_TAGS['positive'])
        
        # 4. Сравнение с конкурентами
        if self.competitors:
            for competitor in self.competitors:
                if competitor in text_lower:
                    tags.append(self.UNIVERSAL_TAGS['comparison'])
                    break
        
        # 5. Вопросы
        if '?' in text or any(word in text_lower for word in ['как', 'что', 'где', 'подскажите']):
            tags.append(self.UNIVERSAL_TAGS['question'])
        
        # 6. Цена
        if re.search(r'цен|стоимост|дорог|дешев', text_lower):
            tags.append(self.UNIVERSAL_TAGS['price'])
        
        # 7. Сервис
        if re.search(r'сервис|обслуживан|персонал|консультац', text_lower):
            tags.append(self.UNIVERSAL_TAGS['service'])
        
        # 8. Технические вопросы
        if re.search(r'техническ|диагностик|ремонт|двигател|бензин|топлив', text_lower):
            tags.append(self.UNIVERSAL_TAGS['technical'])
        
        # Уникальные теги
        unique_tags = []
        for tag in tags:
            if tag not in unique_tags:
                unique_tags.append(tag)
        
        return unique_tags[:5]
    
    def is_dangerous(self, text: str, sentiment: str, tags: List[str]) -> bool:
        """Определение опасности для репутации"""
        text_lower = text.lower()
        danger_score = 0
        
        # 1. Негативная тональность
        if sentiment == 'Негативная':
            danger_score += 2
        
        # 2. Жалоба в тегах
        if self.UNIVERSAL_TAGS['complaint'] in tags:
            danger_score += 2
        
        # 3. Риск-слова
        for risk_word in self.risk_words:
            if risk_word in text_lower:
                danger_score += 1
                if danger_score >= 3:  # Ускоренный выход
                    return True
        
        # 4. Призывы к действию
        if re.search(r'не\s+покупай|не\s+пользуй|бойкотир|распространи|предупреди', text_lower):
            danger_score += 2
        
        # Опасным считается сообщение с 3+ баллами
        return danger_score >= 3
    
    def analyze_mention(self, text: str) -> Dict:
        """Анализ одного упоминания"""
        self.stats['total_mentions'] += 1
        
        # Проверка релевантности
        relevant = self.is_relevant(text)
        if not relevant:
            return {
                'relevant': False,
                'sentiment': 'Нерелевантно',
                'tags': [],
                'dangerous': False
            }
        
        self.stats['relevant_mentions'] += 1
        
        # Анализ тональности
        sentiment_result = self.sentiment_analyzer.analyze(text)
        sentiment = sentiment_result['sentiment']
        source = sentiment_result.get('source', 'unknown')
        
        # Статистика источников
        if source not in self.stats['sentiment_sources']:
            self.stats['sentiment_sources'][source] = 0
        self.stats['sentiment_sources'][source] += 1
        
        if sentiment == 'Негативная':
            self.stats['negative_mentions'] += 1
        
        # Извлечение тегов
        tags = self.extract_tags(text)
        
        # Оценка опасности
        dangerous = self.is_dangerous(text, sentiment, tags)
        if dangerous:
            self.stats['dangerous_mentions'] += 1
        
        return {
            'relevant': True,
            'sentiment': sentiment,
            'tags': tags,
            'dangerous': dangerous,
            'confidence': sentiment_result.get('confidence', 0.5),
            'source': source
        }

In [15]:
class ExcelProcessor:
    """Обработчик файлов"""
    
    def __init__(self, analyzer: MentionAnalyzer):
        self.analyzer = analyzer
    
    def process_file(self, input_path: str, output_path: str):
        """Обработка файла"""
        print(f"\n{'='*60}")
        print(f"Обработка файла: {input_path}")
        print(f"{'='*60}")
        
        # Чтение файла
        df = FileReader.read_file(input_path)
        
        if df.empty:
            print("Файл пуст или не содержит данных")
            return None
        
        print(f"Найдено {len(df)} строк для анализа")
        
        # Анализ каждой строки
        results = []
        from tqdm import tqdm
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Анализ"):
            text = row.get('Текст', '')
            
            if pd.isna(text) or not isinstance(text, str):
                result = {
                    'Сообщение релевантно?': 'Нет',
                    'Тональность': 'Нет данных',
                    'Опасно для репутации': 'Нет'
                }
            else:
                analysis = self.analyzer.analyze_mention(str(text))
                
                result = {
                    'Сообщение релевантно?': 'Да' if analysis['relevant'] else 'Нет',
                    'Тональность': analysis['sentiment'],
                    'Опасно для репутации': 'Да' if analysis['dangerous'] else 'Нет'
                }
            
            # Заполняем теги
            tags = analysis.get('tags', []) if 'analysis' in locals() else []
            for i in range(1, 6):
                if i <= len(tags):
                    result[f'Тег {i}'] = tags[i-1]
                else:
                    result[f'Тег {i}'] = ''
            
            results.append(result)
        
        # Создаем DataFrame с результатами
        results_df = pd.DataFrame(results)
        
        # Объединяем с исходными данными
        output_df = pd.concat([df, results_df], axis=1)
        
        # Сохраняем результат
        return self.save_results(output_df, output_path)
    
    def save_results(self, df: pd.DataFrame, output_path: str) -> str:
        """Сохранение результатов"""
        try:
            # Определяем формат по расширению
            if output_path.endswith('.xlsx') or output_path.endswith('.xls'):
                df.to_excel(output_path, index=False, engine='openpyxl')
                print(f"✓ Результаты сохранены в Excel: {output_path}")
            else:
                # Сохраняем в CSV по умолчанию
                if not output_path.endswith('.csv'):
                    output_path += '.csv'
                df.to_csv(output_path, index=False, encoding='utf-8-sig', sep=';')
                print(f"✓ Результаты сохранены в CSV: {output_path}")
            
            return output_path
            
        except Exception as e:
            print(f"Ошибка сохранения: {e}")
            
            # Пробуем сохранить как CSV с другой кодировкой
            try:
                csv_path = output_path.replace('.xls', '_backup.csv').replace('.xlsx', '_backup.csv')
                df.to_csv(csv_path, index=False, encoding='utf-8', sep=',')
                print(f"✓ Резервная копия сохранена в: {csv_path}")
                return csv_path
            except Exception as e2:
                print(f"Не удалось сохранить даже резервную копию: {e2}")
                return None
    
    def print_statistics(self):
        """Вывод статистики"""
        stats = self.analyzer.stats
        
        print("\n" + "="*60)
        print("СТАТИСТИКА АНАЛИЗА")
        print("="*60)
        print(f"Всего упоминаний: {stats['total_mentions']}")
        
        if stats['total_mentions'] > 0:
            relevant_pct = (stats['relevant_mentions'] / stats['total_mentions']) * 100
            print(f"Релевантных: {stats['relevant_mentions']} ({relevant_pct:.1f}%)")
            print(f"Негативных: {stats['negative_mentions']}")
            print(f"Опасных для репутации: {stats['dangerous_mentions']}")
            
            if stats['sentiment_sources']:
                print("\nИспользованные модели анализа тональности:")
                for source, count in stats['sentiment_sources'].items():
                    pct = (count / stats['relevant_mentions']) * 100 if stats['relevant_mentions'] > 0 else 0
                    print(f"  {source}: {count} ({pct:.1f}%)")
        
        print("="*60)

In [16]:
# Конфигурации для брендов
def get_config(brand_name: str) -> Dict:
    """Получение конфигурации для бренда"""
    
    if brand_name == 'Газпром нефть':
        return {
            'object_name': 'Газпром нефть',
            'keywords': [
                'газпромнефть', 'газпром', 'азс газпром', 
                'гпн', 'gazprom', 'gazpromneft', 'gdrive',
                'газпром нефть', 'газпромнефти'
            ],
            'risk_words': [
                'бадяж', 'некачествен', 'плох', 'ужас', 'кошмар',
                'обман', 'развод', 'жульнич', 'брак', 'испорч',
                'грязн', 'вредн', 'опасн', 'трясет', 'дергает',
                'дым', 'троит', 'глохнет', 'яма', 'разбит'
            ],
            'positive_words': [
                'хорош', 'отличн', 'прекрасн', 'замечательн',
                'рекомендую', 'советую', 'спасибо', 'благодарю',
                'доволен', 'рад', 'класс', 'супер', 'лучш'
            ],
            'competitors': [
                'лукойл', 'роснефть', 'татнефть', 'shell', 'бп',
                'energy', 'энергия', 'аи', 'нефтемагистраль'
            ],
            'use_dostoevsky': False  # Поскольку Dostoevsky не работает
        }
    
    elif brand_name == 'Малютка':
        return {
            'object_name': 'Малютка',
            'keywords': [
                'малютка', 'nutricia', 'нутриция', 'малютк',
                'malyutka', 'детск', 'питани', 'смес', 'каш'
            ],
            'risk_words': [
                'запор', 'колики', 'сыпь', 'рвота', 'аллерги',
                'плох', 'ужас', 'кошмар', 'некачествен', 'вредн'
            ],
            'positive_words': [
                'вкусн', 'нравится', 'люблю', 'обожаю', 'отличн',
                'хорош', 'рекомендую', 'спасибо', 'доверяю'
            ],
            'competitors': [
                'нутрилак', 'нестле', 'хипп', 'семпер', 'фрисо'
            ],
            'use_dostoevsky': False
        }
    
    else:
        # Общая конфигурация по умолчанию
        return {
            'object_name': brand_name,
            'keywords': [brand_name.lower()],
            'risk_words': ['плох', 'некачествен', 'жалоб', 'проблем'],
            'positive_words': ['хорош', 'отличн', 'рекомендую', 'спасибо'],
            'competitors': [],
            'use_dostoevsky': False
        }

In [17]:
def main():
    """Основная функция"""
    print("="*60)
    print("АВТОМАТИЧЕСКИЙ АНАЛИЗАТОР УПОМЕНАНИЙ")
    print("="*60)
    
    # Определяем бренд
    brand_name = 'Газпром нефть'
    print(f"Бренд для анализа: {brand_name}")
    
    # Получаем конфигурацию
    config = get_config(brand_name)
    
    # Создаем анализатор
    analyzer = MentionAnalyzer(config)
    
    # Обрабатываем файл
    input_file = 'date_req.xls'
    
    # Проверяем существование файла
    if not os.path.exists(input_file):
        print(f"\n⚠ Файл не найден: {input_file}")
        print("Используем тестовые данные...")
        
        # Создаем тестовый DataFrame
        test_data = [
            "АЗС Газпром на Щербакова 2. Видимо опять начали бадяжить бензин...",
            "Ребята нужна помощь, человек остался без денег, стоит на заправке Газпромнефть",
            "У тебя даже на крышке бака написано, что минимум 95. Я в свой фф3 лью АИ-100 на газпроме, очень хорошо машине",
            "смех, газпромнефть сам себя проверяет на вшивость)",
            "Хорошее топливо. Хотя качество бензина жалуются."
        ]
        
        df = pd.DataFrame({'Текст': test_data})
        input_file = 'test_data.csv'
        df.to_csv(input_file, index=False, encoding='utf-8')
        print(f"Создан тестовый файл: {input_file}")
    
    output_file = 'date_res.xlsx'
    
    # Обработка
    processor = ExcelProcessor(analyzer)
    result_file = processor.process_file(input_file, output_file)
    
    # Статистика
    processor.print_statistics()
    
    if result_file:
        print(f"\n✓ Анализ завершен успешно!")
        print(f"✓ Результаты сохранены в: {result_file}")
        
        # Показываем первые результаты
        try:
            result_df = pd.read_excel(result_file) if result_file.endswith('.xlsx') else pd.read_csv(result_file, sep=';')
            print("\nПервые 5 строк результатов:")
            print(result_df.head().to_string(index=False))
        except:
            pass
    else:
        print("\n✗ В процессе анализа возникли ошибки")
    
    print("\n" + "="*60)
    print("Рекомендации:")
    print("1. Если Dostoevsky не работает, используйте Transformers (ruBERT)")
    print("2. Для чтения файлов используйте openpyxl для .xlsx и xlrd для .xls")
    print("3. Для CSV файлов определите кодировку с помощью chardet")
    print("="*60)


if __name__ == "__main__":
    main()

АВТОМАТИЧЕСКИЙ АНАЛИЗАТОР УПОМЕНАНИЙ
Бренд для анализа: Газпром нефть

Инициализация анализатора для: Газпром нефть

Инициализация моделей анализа тональности...
  Загрузка Transformers (ruBERT)...


Loading weights: 100%|██████████| 201/201 [00:00<00:00, 629.98it/s, Materializing param=classifier.weight]                                      
BertForSequenceClassification LOAD REPORT from: blanchefort/rubert-base-cased-sentiment
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


  ✓ Transformers: основная модель загружена
✓ Доступно моделей: 2 (transformers, basic)
  Порядок приоритета: transformers, basic

Обработка файла: date_req.xls
Чтение файла: date_req.xls
Определен формат: excel
openpyxl не сработал: File contains no valid workbook part
✓ Прочитано с помощью xlrd
✓ Найдена колонка с текстом: 'Текст'
✓ Успешно прочитано 11 строк
Найдено 11 строк для анализа


Анализ: 100%|██████████| 11/11 [00:01<00:00,  9.98it/s]

✓ Результаты сохранены в Excel: date_res.xlsx

СТАТИСТИКА АНАЛИЗА
Всего упоминаний: 11
Релевантных: 10 (90.9%)
Негативных: 4
Опасных для репутации: 1

Использованные модели анализа тональности:
  transformers: 10 (100.0%)

✓ Анализ завершен успешно!
✓ Результаты сохранены в: date_res.xlsx

Первые 5 строк результатов:
 Unnamed: 0                                                                                                                                                                                                                                                                                                                                                                                                                                       Текст Сообщение релевантно? Тональность Опасно для репутации                    Тег 1                    Тег 2          Тег 3               Тег 4  Тег 5
          1 АЗС Газпром на Щербакова 2. Видимо опять начали бадяжить бензин... Две недели наза